# Exploring Batfish — AVD cEOS Lab

Batfish is a **network configuration analysis tool**. It parses your device configs offline (no live devices needed) and lets you query them like a database.

**Prerequisites:**
1. Batfish container running: `docker run -d -p 9996:9996 batfish/allinone`
2. Configs built: `make build` (generates `arista-avd-lab/intended/configs/*.cfg`)
3. pybatfish installed: `pip install pybatfish jupyter`

---
## How Batfish organises data

```
Batfish server
 └── Network  (logical grouping, e.g. "avd-lab")
      └── Snapshot  (a set of configs at a point in time, e.g. "snapshot")
           └── Queries run against the snapshot
```

Every query returns a **pandas DataFrame** — filter, sort, and export it like any other table.

## 1. Connect and load snapshot

In [3]:
import logging
import shutil
import tempfile
from pathlib import Path

from pybatfish.client.session import Session

# Silence the noisy pybatfish INFO logs
logging.getLogger("pybatfish").setLevel(logging.ERROR)

# Point at the AVD-generated configs (relative to this notebook)
CONFIGS_DIR = Path("../intended/configs")

print(f"Config files found: {sorted(p.name for p in CONFIGS_DIR.glob('*.cfg'))}")

Config files found: ['leaf1.cfg', 'leaf2.cfg', 'leaf3.cfg', 'leaf4.cfg', 'spine1.cfg', 'spine2.cfg']


In [4]:
# Batfish requires a snapshot root containing a configs/ subdirectory
snap_dir = tempfile.mkdtemp(prefix="batfish-snap-")
snap_configs = Path(snap_dir) / "configs"
snap_configs.mkdir()
for cfg in CONFIGS_DIR.glob("*.cfg"):
    shutil.copy2(cfg, snap_configs / cfg.name)

bf = Session(host="localhost")
bf.set_network("avd-lab")
bf.init_snapshot(snap_dir, name="snapshot", overwrite=True)

# Clean up temp dir — Batfish has already uploaded the data to its server
shutil.rmtree(snap_dir)

print("Snapshot loaded.")
print(f"Network : {bf.network}")
print(f"Snapshot: {bf.snapshot}")

Snapshot loaded.
Network : avd-lab
Snapshot: snapshot


> **Tip:** If you already ran `make batfish` or the pipeline with `--batfish`, the snapshot is already on the server. Skip the upload and just do:
> ```python
> bf = Session(host="localhost")
> bf.set_network("avd-lab")
> bf.set_snapshot("snapshot")
> ```

---
## 2. initIssues — parse problems when loading configs

This shows what Batfish **couldn't parse**. For cEOS AVD configs you always get warnings because Batfish's EOS support is incomplete — `vrf instance`, address-family BGP commands, etc. are not fully recognised.

- **Parse warning** = Batfish skipped a line it doesn't understand (safe to ignore for cEOS)
- **Parse error** = Batfish threw an internal exception (also a Batfish bug for cEOS, not your config)

In [5]:
df_init = bf.q.initIssues().answer().frame()
print(f"Total rows: {len(df_init)}")
df_init.head()

Total rows: 123


,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context
0,None,"[configs/leaf1.cfg:[56], configs/leaf2.cfg:[56...",Parse warning,This syntax is unrecognized,vrf instance VRF10,[s_vrf_definition stanza cisco_configuration]
1,None,"[configs/leaf4.cfg:[345, 355]]",Parse warning,This syntax is unrecognized,router-id 10.255.0.6,[s_vrf_definition stanza cisco_configuration]
2,None,[configs/leaf3.cfg:[348]],Parse warning,This syntax is unrecognized,neighbor 10.255.1.101 description leaf4_Vlan3009,[s_vrf_definition stanza cisco_configuration]
3,None,[configs/leaf1.cfg:[289]],Parse warning,This syntax is unrecognized,rd 10.255.0.3:10011,[s_vlan_cisco stanza cisco_configuration]
4,None,[configs/leaf4.cfg:[293]],Parse warning,This syntax is unrecognized,neighbor 10.255.1.100 peer group MLAG-IPv4-UND...,[router_bgp_stanza_tail router_bgp_stanza stan...


In [6]:
# Break down by Type
df_init["Type"].value_counts()

Type
Parse warning    122
Parse error        1
Name: count, dtype: int64

In [7]:
# See only parse errors (Batfish internal bugs with EOS)
df_init[df_init["Type"] == "Parse error"][["Source_Lines", "Details"]].head()

,Source_Lines,Details
122,"[configs/spine1.cfg:[127], configs/spine2.cfg:...",java.lang.NullPointerException: Cannot invoke ...


In [8]:
# See warnings — what EOS syntax Batfish doesn't know about
df_init[df_init["Type"] == "Parse warning"][["Source_Lines", "Line_Text", "Parser_Context"]].drop_duplicates("Line_Text").head(20)

,Source_Lines,Line_Text,Parser_Context
0,"[configs/leaf1.cfg:[56], configs/leaf2.cfg:[56...",vrf instance VRF10,[s_vrf_definition stanza cisco_configuration]
1,"[configs/leaf4.cfg:[345, 355]]",router-id 10.255.0.6,[s_vrf_definition stanza cisco_configuration]
2,[configs/leaf3.cfg:[348]],neighbor 10.255.1.101 description leaf4_Vlan3009,[s_vrf_definition stanza cisco_configuration]
3,[configs/leaf1.cfg:[289]],rd 10.255.0.3:10011,[s_vlan_cisco stanza cisco_configuration]
4,[configs/leaf4.cfg:[293]],neighbor 10.255.1.100 peer group MLAG-IPv4-UND...,[router_bgp_stanza_tail router_bgp_stanza stan...
5,"[configs/leaf1.cfg:[328], configs/leaf2.cfg:[3...",route-target import evpn 10:10,[route_target vrfd_route_target s_vrf_definiti...
6,"[configs/leaf1.cfg:[315], configs/leaf2.cfg:[3...",route-target both 13402:13402,[s_vlan_cisco stanza cisco_configuration]
7,"[configs/leaf1.cfg:[204], configs/leaf2.cfg:[2...",vxlan vrf VRF11 vni 11,[cisco_configuration]
8,"[configs/leaf2.cfg:[332, 342]]",neighbor 10.255.1.96 peer group MLAG-IPv4-UNDE...,[s_vrf_definition stanza cisco_configuration]
9,[configs/leaf2.cfg:[333]],neighbor 10.255.1.96 description leaf1_Vlan3009,[s_vrf_definition stanza cisco_configuration]


---
## 3. nodeProperties — inspect what Batfish understood about each device

In [9]:
df_nodes = bf.q.nodeProperties().answer().frame()
print(f"Nodes in snapshot: {df_nodes['Node'].tolist()}")
df_nodes[["Node", "AS_Path_Access_Lists", "Configuration_Format", "VRFs"]].head(10)

Nodes in snapshot: ['leaf1', 'leaf3', 'leaf2', 'leaf4']


,Node,AS_Path_Access_Lists,Configuration_Format,VRFs
0,leaf1,[],CISCO_IOS,"['MGMT', 'VRF10', 'VRF11', 'default']"
1,leaf3,[],CISCO_IOS,"['MGMT', 'VRF10', 'VRF11', 'default']"
2,leaf2,[],CISCO_IOS,"['MGMT', 'VRF10', 'VRF11', 'default']"
3,leaf4,[],CISCO_IOS,"['MGMT', 'VRF10', 'VRF11', 'default']"


In [10]:
# Which VRFs does leaf1 have?
leaf1 = df_nodes[df_nodes["Node"] == "leaf1"].iloc[0]
print("leaf1 VRFs:", leaf1["VRFs"])

leaf1 VRFs: ['MGMT', 'VRF10', 'VRF11', 'default']


---
## 4. interfaceProperties — every interface Batfish parsed

In [11]:
df_ifaces = bf.q.interfaceProperties().answer().frame()
print(f"Total interfaces: {len(df_ifaces)}")
df_ifaces.columns.tolist()

Total interfaces: 84


['Interface',
 'Access_VLAN',
 'Active',
 'Admin_Up',
 'All_Prefixes',
 'Allowed_VLANs',
 'Auto_State_VLAN',
 'Bandwidth',
 'Blacklisted',
 'Channel_Group',
 'Channel_Group_Members',
 'DHCP_Relay_Addresses',
 'Declared_Names',
 'Description',
 'Encapsulation_VLAN',
 'HSRP_Groups',
 'HSRP_Version',
 'Inactive_Reason',
 'Incoming_Filter_Name',
 'MLAG_ID',
 'MTU',
 'Native_VLAN',
 'Outgoing_Filter_Name',
 'PBR_Policy_Name',
 'Primary_Address',
 'Primary_Network',
 'Proxy_ARP',
 'Rip_Enabled',
 'Rip_Passive',
 'Spanning_Tree_Portfast',
 'Speed',
 'Switchport',
 'Switchport_Mode',
 'Switchport_Trunk_Encapsulation',
 'VRF',
 'VRRP_Groups',
 'Zone_Name']

In [12]:
# Show all interfaces with their primary IP
df_ifaces[["Interface", "Primary_Address", "Description", "Active", "VRF"]].dropna(subset=["Primary_Address"]).head(20)

,Interface,Primary_Address,Description,Active,VRF
3,leaf1[Ethernet10],10.255.255.1/31,P2P_spine1_Ethernet3,True,default
4,leaf1[Ethernet11],10.255.255.3/31,P2P_spine2_Ethernet3,True,default
5,leaf1[Loopback0],10.255.0.3/32,ROUTER_ID,True,default
6,leaf1[Loopback1],10.255.1.3/32,VXLAN_TUNNEL_SOURCE,True,default
7,leaf1[Loopback10],10.255.10.3/32,DIAG_VRF_VRF10,True,VRF10
8,leaf1[Loopback11],10.255.11.3/32,DIAG_VRF_VRF11,True,VRF11
9,leaf1[Management0],172.20.10.12/24,OOB_MANAGEMENT,False,MGMT
12,leaf1[Vlan11],10.10.11.1/24,VRF10_VLAN11,True,VRF10
13,leaf1[Vlan12],10.10.12.1/24,VRF10_VLAN12,True,VRF10
14,leaf1[Vlan21],10.10.21.1/24,VRF11_VLAN21,True,VRF11


In [13]:
# Filter to just leaf1
df_ifaces[df_ifaces["Interface"].astype(str).str.startswith("leaf1:")][["Interface", "Primary_Address", "Description", "Active"]]

,Interface,Primary_Address,Description,Active


---
## 5. bgpSessionCompatibility — are BGP peers properly matched?

Possible `Configured_Status` values:

| Status | Meaning |
|--------|--------|
| `UNIQUE_MATCH` | Both sides configured correctly — session will establish |
| `HALF_OPEN` | This side configured, no matching remote found in snapshot |
| `NO_LOCAL_IP` | Batfish can't determine this router's source IP for the session |
| `UNKNOWN_REMOTE` | Remote IP found but Batfish can't match it to a node |
| `NO_MATCH_FOUND` | Remote IP not present anywhere in the snapshot — real error |
| `MULTIPLE_MATCHES` | Ambiguous — real config problem |

For cEOS AVD, `NO_LOCAL_IP` and `UNKNOWN_REMOTE` are Batfish parser limitations with EOS peer-group BGP syntax, not real problems.

In [14]:
df_bgp = bf.q.bgpSessionCompatibility().answer().frame()
print(f"Total BGP session entries: {len(df_bgp)}")
df_bgp["Configured_Status"].value_counts()

Total BGP session entries: 16


Configured_Status
NO_LOCAL_IP       8
UNKNOWN_REMOTE    8
Name: count, dtype: int64

In [15]:
# Show all rows — what sessions exist and their status
df_bgp[["Node", "VRF", "Local_AS", "Local_IP", "Remote_AS", "Remote_Node", "Remote_IP", "Session_Type", "Configured_Status"]]

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
0,leaf1,default,65101,None,65100,None,10.255.0.2,EBGP_SINGLEHOP,NO_LOCAL_IP
1,leaf1,default,65101,10.255.255.3,65100,None,10.255.255.2,EBGP_SINGLEHOP,UNKNOWN_REMOTE
2,leaf1,default,65101,None,65100,None,10.255.0.1,EBGP_SINGLEHOP,NO_LOCAL_IP
3,leaf1,default,65101,10.255.255.1,65100,None,10.255.255.0,EBGP_SINGLEHOP,UNKNOWN_REMOTE
4,leaf2,default,65101,10.255.255.7,65100,None,10.255.255.6,EBGP_SINGLEHOP,UNKNOWN_REMOTE
5,leaf2,default,65101,10.255.255.5,65100,None,10.255.255.4,EBGP_SINGLEHOP,UNKNOWN_REMOTE
6,leaf2,default,65101,None,65100,None,10.255.0.2,EBGP_SINGLEHOP,NO_LOCAL_IP
7,leaf2,default,65101,None,65100,None,10.255.0.1,EBGP_SINGLEHOP,NO_LOCAL_IP
8,leaf3,default,65102,10.255.255.11,65100,None,10.255.255.10,EBGP_SINGLEHOP,UNKNOWN_REMOTE
9,leaf3,default,65102,10.255.255.9,65100,None,10.255.255.8,EBGP_SINGLEHOP,UNKNOWN_REMOTE


In [16]:
# Filter to only genuinely problematic statuses (not cEOS false positives)
ceos_expected = {"HALF_OPEN", "NO_LOCAL_IP", "UNKNOWN_REMOTE"}
problems = df_bgp[~df_bgp["Configured_Status"].astype(str).isin(ceos_expected | {"UNIQUE_MATCH"})]
if problems.empty:
    print("No genuinely problematic BGP sessions found.")
else:
    problems[["Node", "VRF", "Remote_IP", "Session_Type", "Configured_Status"]]

No genuinely problematic BGP sessions found.


---
## 6. undefinedReferences — things referenced but never defined

This catches route-maps, prefix-lists, peer-groups, ACLs that are **referenced** in the config but **never defined**. For cEOS AVD, BGP peer-group types are false positives because Batfish's EOS parser doesn't recognise the EOS peer-group definition syntax.

In [17]:
df_undef = bf.q.undefinedReferences().answer().frame()
print(f"Total undefined references: {len(df_undef)}")
df_undef["Struct_Type"].value_counts()

Total undefined references: 16


Struct_Type
undeclared bgp peer-group    8
bgp peer-group               4
undeclared bgp peer          4
Name: count, dtype: int64

In [18]:
# Show everything
df_undef[["File_Name", "Struct_Type", "Ref_Name", "Context"]]

,File_Name,Struct_Type,Ref_Name,Context
0,configs/leaf1.cfg,bgp peer-group,MLAG-IPv4-UNDERLAY-PEER,bgp neighbor statement
1,configs/leaf1.cfg,undeclared bgp peer,10.255.1.97 (VRF default),bgp neighbor without remote-as
2,configs/leaf1.cfg,undeclared bgp peer-group,EVPN-OVERLAY-PEERS,bgp peer-group referenced before defined
3,configs/leaf1.cfg,undeclared bgp peer-group,IPv4-UNDERLAY-PEERS,bgp peer-group referenced before defined
4,configs/leaf2.cfg,bgp peer-group,MLAG-IPv4-UNDERLAY-PEER,bgp neighbor statement
5,configs/leaf2.cfg,undeclared bgp peer,10.255.1.96 (VRF default),bgp neighbor without remote-as
6,configs/leaf2.cfg,undeclared bgp peer-group,EVPN-OVERLAY-PEERS,bgp peer-group referenced before defined
7,configs/leaf2.cfg,undeclared bgp peer-group,IPv4-UNDERLAY-PEERS,bgp peer-group referenced before defined
8,configs/leaf3.cfg,bgp peer-group,MLAG-IPv4-UNDERLAY-PEER,bgp neighbor statement
9,configs/leaf3.cfg,undeclared bgp peer,10.255.1.101 (VRF default),bgp neighbor without remote-as


In [19]:
# Filter out BGP false positives — what's left are real missing definitions
real_issues = df_undef[~df_undef["Struct_Type"].astype(str).str.lower().str.contains("bgp")]
if real_issues.empty:
    print("No non-BGP undefined references — configs look clean.")
else:
    real_issues[["File_Name", "Struct_Type", "Ref_Name", "Context"]]

No non-BGP undefined references — configs look clean.


---
## 7. routes — routing table as Batfish models it

Batfish computes the **data-plane** from configs alone (no live devices). This gives you the modelled RIB.

In [20]:
df_routes = bf.q.routes().answer().frame()
print(f"Total route entries across all nodes: {len(df_routes)}")
df_routes[["Node", "VRF", "Network", "Next_Hop_IP", "Protocol", "Admin_Distance", "Metric"]].head(20)

Total route entries across all nodes: 96


,Node,VRF,Network,Next_Hop_IP,Protocol,Admin_Distance,Metric
0,leaf1,VRF10,10.10.11.0/24,AUTO/NONE(-1l),connected,0,0
1,leaf1,VRF10,10.10.11.1/32,AUTO/NONE(-1l),local,0,0
2,leaf1,VRF10,10.10.12.0/24,AUTO/NONE(-1l),connected,0,0
3,leaf1,VRF10,10.10.12.1/32,AUTO/NONE(-1l),local,0,0
4,leaf1,VRF10,10.255.1.96/31,AUTO/NONE(-1l),connected,0,0
5,leaf1,VRF10,10.255.1.96/32,AUTO/NONE(-1l),local,0,0
6,leaf1,VRF10,10.255.10.3/32,AUTO/NONE(-1l),connected,0,0
7,leaf1,VRF11,10.10.21.0/24,AUTO/NONE(-1l),connected,0,0
8,leaf1,VRF11,10.10.21.1/32,AUTO/NONE(-1l),local,0,0
9,leaf1,VRF11,10.10.22.0/24,AUTO/NONE(-1l),connected,0,0


In [ ]:
# Routes on leaf1 in the default VRF
df_routes[(df_routes["Node"] == "leaf1") & (df_routes["VRF"] == "default")][
    ["Network", "Next_Hop_IP", "Protocol", "Admin_Distance", "Metric"]
].sort_values("Network")

In [ ]:
# What protocols are present on leaf1?
df_routes[df_routes["Node"] == "leaf1"]["Protocol"].value_counts()

In [ ]:
# Find all nodes that have a route to the loopback prefix 10.255.0.0/24
df_routes[df_routes["Network"].astype(str).str.startswith("10.255.0.")][
    ["Node", "VRF", "Network", "Protocol", "Next_Hop_IP"]
].sort_values(["Node", "Network"])

---
## 8. bgpRoutes — the BGP RIB

Shows what BGP routes each node has — the protocol-specific RIB before route selection.

In [ ]:
df_bgp_routes = bf.q.bgpRoutes().answer().frame()
print(f"Total BGP RIB entries: {len(df_bgp_routes)}")
df_bgp_routes.columns.tolist()

In [ ]:
df_bgp_routes[["Node", "VRF", "Network", "Next_Hop_IP", "AS_Path", "Local_Pref", "Communities"]].head(20)

---
## 9. Saving results to CSV

Any DataFrame can be exported:

In [ ]:
out_dir = Path("reports")
out_dir.mkdir(exist_ok=True)

df_init.to_csv(out_dir / "init_issues.csv",    index=False)
df_bgp.to_csv( out_dir / "bgp_sessions.csv",   index=False)
df_undef.to_csv(out_dir / "undefined_refs.csv", index=False)
df_routes.to_csv(out_dir / "routes.csv",        index=False)

print(f"CSVs written to {out_dir.resolve()}/")

---
## 10. Listing available questions

Batfish has ~60 built-in questions. Here's how to see them all:

In [ ]:
# Every question available on this Batfish server
questions = [q for q in dir(bf.q) if not q.startswith("_")]
print(f"Total questions: {len(questions)}")
print("\n".join(sorted(questions)))

In [ ]:
# Read the docstring for any question before running it
help(bf.q.routes)